In [ ]:
#installs
!pip install langchain openai stability-sdk pillow
!pip install langchain-community langchain-core
!pip install wikipedia-api transformers
!pip install wikipedia
!pip install openai==0.28

In [ ]:
#imports
import wikipediaapi
from transformers import pipeline, CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
import math
import json
import io
import os
import warnings
import random
import torch
import re
from PIL import Image, ImageDraw, ImageFont
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
import langchain
import openai
import stability_sdk
import wikipedia

In [ ]:
# API Keys
os.environ['OPENAI_API_KEY'] = 'sk-'
os.environ['STABILITY_KEY'] = 'sk-'
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
# Template Comic Prompt
template = """
You are a cartoon creator.

You will be given a scenario, you must split it in 6 parts.
Please split it into 6 parts as this will feed into a comic book format.

Each part will be a different cartoon panel.
For each cartoon panel, you will write a description of it with:
 - the characters in the panel, they must be described precisely each time
 - the background of the panel
The description should be only word or group of word delimited by a comma, no sentence.
Always use the characters descriptions instead of their name in the cartoon panel description.
You can not use the same description twice.
You will also write the text of the panel.
The text should not be more than 2 small sentences.
Each sentence should start by the character name

Example input:
Characters: Adrien is a guy with blond hair wearing glasses. Vincent is a guy with black hair wearing a hat.
Adrien and vincent want to start a new product, and they create it in one night before presenting it to the board.

Example output:

# Panel 1
description: 2 guys, a blond hair guy wearing glasses, a dark hair guy wearing hat, sitting at the office, with computers
text:
```
Vincent: I think Generative AI are the future of the company.
Adrien: Let's create a new product with it.
```
# end

Short Scenario:
{scenario}

Split the scenario in 6 parts:
"
"""

In [ ]:
def search_wikipedia(query):
    search_results = wikipedia.search(query)
    if not search_results:
        return None
    return search_results[0]  # Return the most relevant page title

def get_wikipedia_content(page_title):
    user_agent = 'MyWikiScraper/1.0 (random_email@test.com)'
    wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})
    page = wiki_wiki.page(page_title)
    if not page.exists():
        return None
    return page.text

def identify_relevant_sections(content, query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Extract the sections of the following text that are relevant to the mythology story of '{query}':\n\n{content}\n\nRelevant sections:"}
        ],
        max_tokens=1500,
        temperature=0.5,
    )
    return response.choices[0].message.content.strip()

def split_content(content, max_length=512):
    sentences = content.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > max_length:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks

def summarize_content(content, max_length=500, min_length=150):
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    summary = summarizer(content, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

def extract_characters(summary):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Extract the characters and their descriptions from the following summary:\n\n{summary}\n\nCharacters and descriptions:"}
        ],
        max_tokens=500,
        temperature=0.5,
    )
    character_descriptions = response.choices[0].message.content.strip()
    return character_descriptions

def format_summary_to_template(summary):
    characters = extract_characters(summary)
    if not characters:
        characters = ["A character is present."]

    parts = summary.split('. ')
    split_index = max(1, len(parts) // 6)
    panels = ['. '.join(parts[i:i + split_index]) for i in range(0, len(parts), split_index)]

    formatted_output = (
        "You are a cartoon creator.\n\n"
        "You will be given a short scenario, you must split it in 6 parts.\n"
        "Each part will be a different cartoon panel.\n"
        "For each cartoon panel, you will write a description of it with:\n"
        " - the characters in the panel, they must be described precisely each time\n"
        " - the background of the panel\n"
        " - the action or emotion of the characters\n"
        "The description should be only word or group of words delimited by commas, no sentences.\n"
        "Always use the characters' descriptions instead of their names in the cartoon panel description.\n"
        "You can not use the same description twice.\n"
        "You will also write the text of the panel.\n"
        "The text should not be more than 2 small sentences.\n"
        "Each sentence should start with the character's name\n\n"
        "Example input:\n"
        "Characters: Adrien is a guy with blond hair wearing glasses. Vincent is a guy with black hair wearing a hat.\n"
        "Scenario: Adrien and Vincent want to start a new product, and they create it in one night before presenting it to the board.\n\n"
        "Example output:\n\n"
        "# Panel 1\n"
        "description: 2 guys, blond hair guy wearing glasses, dark hair guy wearing hat, sitting at office, computers, determined expressions\n"
        "text:\n"
        "```\n"
        "Vincent: I think Generative AI is the future of the company.\n"
        "Adrien: Let's create a new product with it.\n"
        "```\n"
        "# end\n\n"
        "Short Scenario:\n{}\n\n"
        "Split the scenario into 6 parts:\n\n".format(summary)
    )

    for i, panel in enumerate(panels, 1):
        formatted_output += f"# Panel {i}\n"
        formatted_output += "description: <description of the panel based on the context>\n"
        formatted_output += "text:\n"
        formatted_output += "```\n"
        formatted_output += f"{characters.splitlines()[i % len(characters.splitlines())]}: {panel}\n"
        formatted_output += "```\n"
        formatted_output += "# end\n\n"

    return formatted_output

def scrape_summarize_wikipedia(query):
    page_title = search_wikipedia(query)
    if page_title is None:
        return "No relevant Wikipedia page found."

    content = get_wikipedia_content(page_title)
    if content is None:
        return "The page does not exist."

    relevant_content = identify_relevant_sections(content, query)
    if not relevant_content:
        return "No relevant mythology content found on the page."

    chunks = split_content(relevant_content, max_length=512)
    summaries = [summarize_content(chunk) for chunk in chunks]
    summary = ' '.join(summaries)

    # Use ChatGPT to generate additional detailed text
    gpt_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Updated to use a supported model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate additional detailed text for the mythology summary:\n\n{summary}\n\nAdditional details:"}
        ],
        max_tokens=300,
        temperature=0.7,
        top_p=1.0,
        stop=None
    )

    detailed_text = gpt_response.choices[0].message.content.strip()

    enhanced_summary = f"{summary} {detailed_text}"

    return enhanced_summary

def format_wikipedia(summary):
    formatted_summary = format_summary_to_template(summary)
    return formatted_summary

In [ ]:
# Function to generate panels from scenario
def generate_panels(scenario):
    model = ChatOpenAI(model_name='gpt-3.5-turbo')

    human_message_prompt = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chat_prompt.format_messages(scenario=scenario)

    result = model(chat_prompt.format_messages(scenario=scenario))

    print(result.content)

    return extract_panel_info(result.content)

def extract_panel_info(text):
    panel_info_list = []
    panel_blocks = text.split('# Panel')

    for block in panel_blocks:
        if block.strip() != '':
            panel_info = {}

            panel_number = re.search(r'\d+', block)
            if panel_number is not None:
                panel_info['number'] = panel_number.group()

            panel_description = re.search(r'description: (.+)', block)
            if panel_description is not None:
                panel_info['description'] = panel_description.group(1)

            panel_text = re.search(r'text:\n```\n(.+)\n```', block, re.DOTALL)
            if panel_text is not None:
                panel_info['text'] = panel_text.group(1)

            panel_info_list.append(panel_info)
    return panel_info_list

def resize_and_add_border(image, target_size, border_size):
    resized_image = Image.new("RGB", target_size, "black")
    resized_image.paste(image, ((target_size[0] - image.width) // 2, (target_size[1] - image.height) // 2))
    return resized_image

def create_strip(images):
    columns, rows = 2, 3

    output_width = columns * images[0].width + (columns - 1) * 10
    output_height = rows * images[0].height + (rows - 1) * 10

    result_image = Image.new("RGB", (output_width, output_height), "white")

    for i, img in enumerate(images):
        x = (i % columns) * (img.width + 10)
        y = (i // columns) * (img.height + 10)

        resized_img = resize_and_add_border(img, (images[0].width, images[0].height), 10)
        result_image.paste(resized_img, (x, y))

    return result_image.resize((1024, 1536))

def add_text_to_panel(text, panel_image):
    text_image = generate_text_image(text)

    result_image = Image.new('RGB', (panel_image.width, panel_image.height + text_image.height))

    result_image.paste(panel_image, (0, 0))

    result_image.paste(text_image, (0, panel_image.height))

    return result_image

def text_to_image(prompt):
    seed = random.randint(0, 1000000000)
    stability_api = client.StabilityInference(
        key=os.environ['STABILITY_KEY'],
        verbose=True,
        engine="stable-diffusion-xl-1024-v1-0",
    )

    answers = stability_api.generate(
        prompt=prompt,
        seed=seed,
        steps=30,
        cfg_scale=8.0,
        width=1024,
        height=1024,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                return img

def generate_text_image(text):
    width = 1024
    height = 128
    image = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype(font="manga-temple.ttf", size=30)
    text_width, text_height = draw.textsize(text, font=font)
    x = (width - text_width) // 2
    y = (height - text_height) // 2
    text_color = (0, 0, 0)
    draw.text((x, y), text, fill=text_color, font=font)
    return image

In [ ]:
# Example usage
query = "Icarus"
summary = scrape_summarize_wikipedia(query)
formatted_summary = format_wikipedia(summary)
print(formatted_summary)

Your max_length is set to 500, but your input_length is only 496. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=248)


You are a cartoon creator.

You will be given a short scenario, you must split it in 6 parts.
Each part will be a different cartoon panel.
For each cartoon panel, you will write a description of it with:
 - the characters in the panel, they must be described precisely each time
 - the background of the panel
 - the action or emotion of the characters
The description should be only word or group of words delimited by commas, no sentences.
Always use the characters' descriptions instead of their names in the cartoon panel description.
You can not use the same description twice.
You will also write the text of the panel.
The text should not be more than 2 small sentences.
Each sentence should start with the character's name

Example input:
Characters: Adrien is a guy with blond hair wearing glasses. Vincent is a guy with black hair wearing a hat.
Scenario: Adrien and Vincent want to start a new product, and they create it in one night before presenting it to the board.

Example output:

#

In [ ]:
SCENARIO = formatted_summary
STYLE = "american comic, colored"

# Generate panels
panels = generate_panels(SCENARIO)

with open('panels.json', 'w') as outfile:
    json.dump(panels, outfile)

# Generate panel images and create a strip
panel_images = []

for panel in panels:
    panel_prompt = panel["description"] + ", cartoon box, " + STYLE
    print(f"Generate panel {panel['number']} with prompt: {panel_prompt}")
    panel_image = text_to_image(panel_prompt)
    panel_image_with_text = add_text_to_panel(panel["text"], panel_image)
    panel_image_with_text.save(f"panel-{panel['number']}.png")
    panel_images.append(panel_image_with_text)

create_strip(panel_images).save("strip.png")

INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443
INFO:stability_sdk.client:Sending request.


# Panel 1
description: Daedalus: master craftsman, architect of the labyrinth of Crete, Icarus: son of Daedalus, sitting at a workshop, crafting wax and feather wings
text:
```
Daedalus: I have crafted these wings for you, my son. Be cautious when you fly.
Icarus: Thank you, father. I will heed your advice.
```
# end

# Panel 2
description: Icarus: wearing wax and feather wings, Daedalus: watching from below, with a worried expression, blue sky with the sun shining
text:
```
Icarus: The sky is so vast and inviting. I must soar higher.
Daedalus: Be careful, my son! Do not fly too close to the sun.
```
# end

# Panel 3
description: Icarus: flying too close to the sun, wings melting, Daedalus: looking up in horror, wings falling apart
text:
```
Icarus: The sun's warmth is exhilarating! I feel alive!
Daedalus: No, Icarus! Your wings are melting! Descend quickly!
```
# end

# Panel 4
description: Icarus: falling from the sky, wings disintegrating, Daedalus: reaching out in desperation, ocea

INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 6.71s
<ipython-input-7-162f8172bbf9>:107: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443
INFO:stability_sdk.client:Sending request.


Generate panel 2 with prompt: Icarus: wearing wax and feather wings, Daedalus: watching from below, with a worried expression, blue sky with the sun shining, cartoon box, american comic, colored


INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 7.11s
<ipython-input-7-162f8172bbf9>:107: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443
INFO:stability_sdk.client:Sending request.


...

INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 6.23s
<ipython-input-7-162f8172bbf9>:107: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443
INFO:stability_sdk.client:Sending request.


Generate panel 5 with prompt: Icarus: plunging into the sea, splashing water, sinking beneath the waves, Daedalus: grieving on the shore, cartoon box, american comic, colored


INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 5.69s
<ipython-input-7-162f8172bbf9>:107: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443
INFO:stability_sdk.client:Sending request.


Generate panel 6 with prompt: Daedalus: mourning his son, looking at the sky, Icarus: spirit soaring above, in peace, cartoon box, american comic, colored


INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 6.23s
<ipython-input-7-162f8172bbf9>:107: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
